In [2]:
import itertools
RRR = RealField(100);

def accprob(transmatrices,sigma,accept_states):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols() == len(accept_states);
    n = transmatrices[0].nrows();
    Id = identity_matrix(n);
    v = Matrix(Id[0]);
    f = Matrix(accept_states).transpose();
    
    A = v*Id;
    for ch in sigma:
        assert int(ch) == 0 or int(ch) == 1;
        A = A*transmatrices[int(ch)];
    A = A*f;
    return A[0][0];

 
def probmatrix(transmatrices,sigma):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols();
    A = identity_matrix(transmatrices[0].nrows());
    for ch in sigma:
        assert int(ch) == 0 or int(ch) == 1;
        A = A*transmatrices[int(ch)];
    return A;
    
def list_probs(transmatrices,accept_states,upto=11,startfrom=1):
    assert transmatrices[0].nrows() == transmatrices[0].ncols() == transmatrices[1].nrows() == transmatrices[1].ncols() == len(accept_states);
    n = transmatrices[0].nrows();
    Id = identity_matrix(n);
    v = Matrix(Id[0]);
    f = Matrix(accept_states).transpose();
    
    problist = []; #list of strings with their acceptance probabilities
    for i in range(startfrom,upto):
        l = list(itertools.product([0,1],repeat=i));
        for st in l:
            A = v*Id;
            for ch in st:
                assert int(ch) == 0 or int(ch) == 1;
                A = A*transmatrices[int(ch)];
            A = A*f;
            problist.append((st,A[0][0]));
        
    return problist;
    
#expects a list in exactly the format returned by list_probs. if length>0, only looks at strings of exactly that length
def highest_prob(problist,length=0):
    assert length >= 0;
    if length == 0:
        sublist = problist;
    elif length > 0:
        sublist = [l for l in problist if len(l[0]) == length];
    highprob = max([l[1] for l in sublist]);
    highstr = [st for st in sublist if st[1] == highprob];
    return highstr;
    
def sortproblist(problist):
    return sorted(problist,key=lambda x:x[1],reverse=True);
    
    
#check transition matrices for rows summing to 1
def checkmatrices(P):
    assert P[0].nrows() == P[0].ncols() == P[1].nrows() == P[1].ncols();
    n = P[0].nrows();
    
    for i in range(2):
        for j in range(n):
            s = sum(P[i][j]);
            if s != 1:
                if s > 1:
                    relto1 = " > 1";
                elif s < 1:
                    relto1 = " < 1";
                print "P"+str(i)+" row "+str(j+1)+" sum is "+str(RRR(s))+relto1;
                
#expects a list in format returned by list_probs; if length>0, only consider strings of that length
def checkstd(problist,length=0):
    assert length >= 0;
    if length == 0:
        sublist = problist;
    elif length > 0:
        sublist = [l for l in problist if len(l[0]) == length];
    std_list = stats.std([p[1] for p in sublist]);
    high = highest_prob(sublist)[0];
    for p in sublist:
        if p[0] != high[0]:
            diff = high[1] - std_list;
            if diff <= p[1]:
                print str(high[0])+" is less than std = "+str(std_list)+" from "+str(p[0])+" (with prob "+str(p[1])+")";

In [3]:
# sigma = 01
eps = RRR(.001);
#AA01 = [Matrix(RRR,[[eps,1-eps],[0,0]]), Matrix(RRR,[[0,0],[eps,1-eps]])];
#AA01 = [Matrix(RRR,[[0,1],[0,0]]), Matrix(RRR,[[0,0],[0,1]])];
AA01 = [Matrix(RRR,[[eps,1-eps],[.5,.5]]), Matrix(RRR,[[.5,.5],[eps,1-eps]])];
checkmatrices(AA01)

l = list_probs(AA01,upto=6);
print highest_prob(l,2)

[((0, 1), 0.99850100000000000000000000000)]

In [4]:
# sigma = 001
eps = RRR(.0000001);
AA001 = [Matrix(RRR,[[1-eps,eps],[.5,.5]]), Matrix(RRR,[[eps,1-eps],[.5,.5]])];
checkmatrices(AA001)

l2 = list_probs(AA001,upto=9);
print highest_prob(l2,3)

[((0, 0, 1), 0.99999982500001999999900000000)]

In [10]:
checkstd(l2,3)

(0, 0, 1) is less than std = 0.32732673715595051322979025720 from (1, 0, 1) (with prob 0.74999997499999000000100000000)
(0, 0, 1) is less than std = 0.32732673715595051322979025720 from (1, 1, 1) (with prob 0.74999992500000999999900000000)

In [9]:
l2_len3 = [p for p in l2 if len(p[0]) == 3];
std_len3 = stats.std([p[1] for p in l2_len3]);
highlen3 = highest_prob(l2_len3)[0];
for p in l2_len3:
    if p[0] != highlen3[0]:
        diff = highlen3[1] - std_len3;
        if diff <= p[1]:
            print str(highlen3[0])+" is less than std = "+str(std_len3)+" from "+str(p[0])+" (with prob "+str(p[1])+")";

(0, 0, 1) is less than std = 0.31763103428978724401977571436 from (1, 0, 1) (with prob 0.74740100000000000000000000000)
(0, 0, 1) is less than std = 0.31763103428978724401977571436 from (1, 1, 1) (with prob 0.74259900000000000000000000000)

In [11]:
checkstd(l2,2)

In [5]:
print highest_prob(l2,8)

[((0, 0, 0, 0, 0, 0, 0, 1), 0.98045731947170510000000000000)]

In [6]:
# sigma = 01101
eps = QQ(.01);
AA3 = [Matrix(QQ,[[eps,1-2*eps,eps],[eps,eps,1-2*eps],[1/3,1/3,1/3]]), Matrix(QQ,[[1/3,1/3,1/3],[eps,1-2*eps,eps],[1-2*eps,eps,eps]])];
checkmatrices(AA3)

l3 = list_probs(AA3,upto=9,halt_last=False);
print highest_prob(l3,5)

[((0, 1, 1, 0, 1), 750275851777/810000000000)]

In [12]:
checkstd(l3,5)

In [7]:
# sigma = 01101, converting an NFA
eps = QQ(.00000001);
#[(1-eps)/2,eps,(1-eps)/2]
#AA4 = [Matrix(QQ,[[eps,1-2*eps,eps],[1/3,1/3,1/3],[eps,1-2*eps,eps]]), Matrix(QQ,[[1/3,1/3,1/3],[1-eps,2*eps,1-eps],[eps,eps,1-2*eps]])];
AA4 = [Matrix(QQ,[[1/3-eps,1/3+2*eps,1/3-eps],[1/3,1/3,1/3],[1/3-eps,1/3+2*eps,1/3-eps]]), Matrix(QQ,[[1/3,1/3,1/3],[(1-eps)/2,eps,(1-eps)/2],[1/3-eps,1/3-eps,1/3+2*eps]])];
checkmatrices(AA4)

l4 = list_probs(AA4,upto=9,halt_last=False);
print highest_prob(l4,5)

[((1, 0, 1, 0, 1), 5249999967500000199999991/13500000000000000000000000)]

In [8]:
# NFA for sigma = 01101
AA4_NFA = [Matrix([[0,1,0],[0,0,0],[0,1,0]]), Matrix([[0,0,0],[1,0,1],[0,0,1]])];

l4_nfa = list_probs(AA4_NFA,[1,0,0],upto=9);
print highest_prob(l4_nfa,5)

[((0, 1, 1, 0, 1), (1))]

In [15]:
# trying to convert leaving 0 rows
eps = QQ(.001); #.0001 fails!
#AA4_zeros = [Matrix(RRR,[[eps,1-2*eps,eps],[0,0,0],[eps,1-2*eps,eps]]), Matrix(RRR,[[0,0,0],[1-eps,2*eps,1-eps],[eps,eps,1-2*eps]])];
#AA4_zeros = [Matrix(RRR,[[1/3-eps,1/3+2*eps,1/3-eps],[0,0,0],[1/3-eps,1/3+2*eps,1/3-eps]]), Matrix(RRR,[[0,0,0],[(1-eps)/2,eps,(1-eps)/2],[1/3-eps,1/3-eps,1/3+2*eps]])];
#AA4_zeros = [Matrix(QQ,[[1/4,1/2,1/4],[0,0,0],[1/4,1/2,1/4]]), Matrix(QQ,[[0,0,0],[3/8,1/4,3/8],[1/4,1/4,1/2]])];
AA4_zeros = [Matrix(QQ,[[1/4,1/2,1/4],[1/3-eps,1/3-eps,1/3-eps],[1/4,1/2,1/4]]), Matrix(QQ,[[1/3-eps,1/3-eps,1/3-eps],[3/8,1/4,3/8],[1/4,1/4,1/2]])];
checkmatrices(AA4_zeros)

l4_zeros = list_probs(AA4_zeros,upto=9,halt_last=False);
print highest_prob(l4_zeros,5)

P0 row 2 sum is 0.99700000000000000000000000000 < 1
P1 row 1 sum is 0.99700000000000000000000000000 < 1
[((0, 1, 1, 0, 1), 26597295404239/81000000000000)]

In [13]:
#sortproblist([p for p in l4_zeros if len(p[0]) == 5])

In [14]:
RRR(3/32-171/2048)

0.010253906250000000000000000000

In [16]:
#checkstd(l4_zeros,5)

In [17]:
# NFA for sigma=1001010
AA5_NFA = [Matrix([[0,0,0,0],[1,0,1,0],[0,0,0,1],[0,0,1,0]]), Matrix([[0,1,0,0],[0,0,0,0],[0,1,0,0],[0,0,0,1]])];

l5_nfa = list_probs(AA5_NFA,[1,0,0,0],startfrom=7,upto=8);
print highest_prob(l5_nfa)

[((1, 0, 0, 1, 0, 1, 0), (1))]

In [18]:
#AA5_zeros = [Matrix(RRR,[[0,0,0,0],[3/8,1/8,3/8,1/8],[1/12,1/12,1/12,3/4],[1/12,1/12,3/4,1/12]]), Matrix(RRR,[[1/12,3/4,1/12,1/12],[0,0,0,0],[1/12,3/4,1/12,1/12],[1/12,1/12,1/12,3/4]])];

eps = RRR(.001);
AA5_zeros = [Matrix(RRR,[[1/4-eps,1/4-eps,1/4-eps,1/4-eps],[3/8,1/8,3/8,1/8],[1/12,1/12,1/12,3/4],[1/12,1/12,3/4,1/12]]), Matrix(RRR,[[1/12,3/4,1/12,1/12],[1/4-eps,1/4-eps,1/4-eps,1/4-eps],[1/12,3/4,1/12,1/12],[1/12,1/12,1/12,3/4]])];

checkmatrices(AA5_zeros)

l5_zeros = list_probs(AA5_zeros,startfrom=7,upto=8,halt_last=False);
print highest_prob(l5_zeros)

P0 row 1 sum is 0.99600000000000000000000000000 < 1
P1 row 1 sum is 1.0000000000000000000000000000 < 1
P1 row 2 sum is 0.99600000000000000000000000000 < 1
P1 row 3 sum is 1.0000000000000000000000000000 < 1
[((1, 0, 0, 1, 0, 1, 0), 0.26418663039191702390306355738)]

In [19]:
# sigma = 0^610^3 NFA
AA6_NFA = [Matrix([[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[1,0,0,0,1],[0,0,0,0,1]]), Matrix([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,1,0,0]])]

#AA6_NFA = [Matrix([[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[.5,0,0,0,.5],[0,0,0,0,1]]), Matrix([[.2,.2,.2,.2,.2],[.2,.2,.2,.2,.2],[.2,.2,.2,.2,.2],[.2,.2,.2,.2,.2],[0,0,1,0,0]])]

l6_nfa = list_probs(AA6_NFA,[0,1,0,0,0],startfrom=10,upto=11);
print highest_prob(l6_nfa)

[((0, 0, 0, 0, 0, 0, 1, 0, 0, 0), (1))]

In [20]:
eps4 = .01;
#eps3 = .01;
eps3 = eps4;
#AA6_zeros = [Matrix(QQ,[[1/6,1/3,1/6,1/6,1/6],[1/6,1/6,1/3,1/6,1/6],[1/6,1/6,1/6,1/3,1/6],[1/3,1/9,1/9,1/9,1/3],[1/6,1/6,1/6,1/6,1/3]]), Matrix(QQ,[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[1/6,1/6,1/3,1/6,1/6]])];
AA6_zeros = [Matrix(RR,[[eps4,1-4*eps4,eps4,eps4,eps4],[eps4,eps4,1-4*eps4,eps4,eps4],[eps4,eps4,eps4,1-4*eps4,eps4],[(1-3*eps3)/2,eps3,eps3,eps3,(1-3*eps3)/2],[eps4,eps4,eps4,eps4,1-4*eps4]]), Matrix(RR,[[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[eps4,eps4,1-4*eps4,eps4,eps4]])];
checkmatrices(AA6_zeros)

l6_zeros = list_probs(AA6_zeros,[0,1,0,0,0],startfrom=10,upto=11);
print highest_prob(l6_zeros)

P1 row 1 sum is 0.00000000000000000000000000000 < 1
P1 row 2 sum is 0.00000000000000000000000000000 < 1
P1 row 3 sum is 0.00000000000000000000000000000 < 1
P1 row 4 sum is 0.00000000000000000000000000000 < 1
[((0, 0, 0, 0, 0, 0, 1, 0, 0, 0), 0.209441250924369)]